In [21]:
import yaml
import os

In [22]:
# Data wrangling
import pandas as pd
import numpy as np

In [23]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm

In [24]:
# Models
from lightgbm import LGBMRegressor

In [25]:
# Models Pipelines
from sklearn.pipeline import Pipeline

In [26]:
#Hyperparameter optimization
import optuna

In [27]:
# Model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error

In [28]:
import mlflow
import mlflow.sklearn

In [29]:
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("distillation_tower")

<Experiment: artifact_location=('file:///c:/Users/mjkipsz2/OneDrive - The University of '
 'Manchester/Desktop/distillation_colum_yield/notebooks/mlruns/550582570200691247'), creation_time=1736352237514, experiment_id='550582570200691247', last_update_time=1736352237514, lifecycle_stage='active', name='distillation_tower', tags={}>

In [30]:
# Save model
import joblib

In [31]:
# Load the configuration file
with open('../config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

# Access the settings
project_folder = config['projectFolder']
df_path = os.path.join(project_folder, config['transformedDataFile'])
df = pd.read_csv(df_path)

# Display the first few rows
df.head()

,PressureC1_diff,FlowC1,Temp1,Yield
0,0.0000,432.0636,139.9857,69.400623
1,-9.9628,487.4029,131.0470,66.532666
2,-0.0695,437.3516,118.2666,71.102193
3,0.2257,481.8314,118.1769,69.793481
4,-0.1678,412.6471,120.7891,71.489516


In [32]:
# Define features (X) and target variable (y)
X = df.drop('Yield', axis=1)  # Features (all columns except 'Job Offer')
y = df['Yield']  # Target variable

In [33]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Check the shape of the data
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (202, 3)
Testing data shape: (51, 3)


In [34]:
def get_metrics(y_true, y_pred):

    dict_metrics = {
        'MSE': mean_squared_error(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'R2 Score': r2_score(y_true, y_pred),
        
    }
    return dict_metrics

In [35]:
# Creating pipeline with LGBM Regressor
pip_model_lgbm = Pipeline(steps=[
    ('model', LGBMRegressor(random_state=2023,force_col_wise=True))
])

# Fit pipeline
pip_model_lgbm.fit(X_train, y_train)

# Generate Predictions using the correctly fitted pipeline
y_pred = pip_model_lgbm.predict(X_test)

# Evaluate Metrics
metrics = get_metrics(y_test, y_pred)

# View Results
metrics

[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 202, number of used features: 3
[LightGBM] [Info] Start training from score 69.521861
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

c:\Users\mjkipsz2\OneDrive - The University of Manchester\Desktop\distillation_colum_yield\.venv\lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning:

The LGBMRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.



{'MSE': 0.8304275732374204,
 'MAE': 0.6985192486419486,
 'R2 Score': 0.5928737530617233}

In [36]:
#Hyperparameter optimization with Optuna

# Define the objective function
def objective(trial):
    # Define the hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 50),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42,
        'force_col_wise': True
    }
    
    # Create and train model
    model = LGBMRegressor(**params)
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate error (you can change this to other metrics)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Create and run study
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42)
)

# Add this to suppress Optuna logging
optuna.logging.set_verbosity(optuna.logging.WARNING)

# When optimizing, disable progress bar if desired
study.optimize(objective, n_trials=50, show_progress_bar=False)

# Train final model with best parameters
best_params = study.best_trial.params
print("Best hyperparameters:", study.best_params)

[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 202, number of used features: 3
[LightGBM] [Info] Start training from score 69.521861
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [37]:
# Start an MLflow run with a specific run name
with mlflow.start_run(run_name="lgbm_regression_run"):
    # Train the final model
    final_model = LGBMRegressor(**best_params)

    final_model.fit(X_train, y_train)

    # Add model signature and input example
    signature = mlflow.models.infer_signature(X_train, y_train)
    input_example = X_train.iloc[:5]

    # Log metrics
    y_pred = final_model.predict(X_test)
    metrics = get_metrics(y_test, y_pred)
    mlflow.log_metrics(metrics)

    # Log parameters
    mlflow.log_params(best_params)

    # Log the model with signature and input example
    mlflow.sklearn.log_model(
        final_model, 
        "light_gbm_model",
        signature=signature,
        input_example=input_example
    )

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000039 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 202, number of used features: 3
[LightGBM] [Info] Start training from score 69.521861
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [38]:
# Visualize the optimization history
optuna.visualization.plot_optimization_history(study).show()

In [39]:
# Visualize parameter importance
optuna.visualization.plot_param_importances(study).show()

In [40]:
#joblib.dump(tuned_model, 'CIC_hybrid_no_outliers_Norsok_scikit.joblib')